In [1]:
#!pip3 install pytorch_pretrained_bert
! pip install transformers

import numpy as np
import pandas as pd

import torch
import sys
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, \
SequentialSampler
from sklearn.metrics import f1_score
from tqdm import trange


from transformers import AutoTokenizer, AutoModel
from torch.optim import Adam
from transformers import get_linear_schedule_with_warmup, AdamW

     |████████████████████████████████| 890kB 8.0MB/s 
     |████████████████████████████████| 890kB 28.9MB/s 
     |████████████████████████████████| 3.0MB 55.8MB/s 
     |████████████████████████████████| 1.1MB 60.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=51416ff651d19c79b32e79bd98f5d2d5aea4252ee1f33e57e5c024256d791abb
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

! cp -r -f /content/drive/'My Drive'/msu/ner /content/
import sys

sys.path.append('/content/msu/ner/')
dir_name = '/content/ner/BC2GM-IOB/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
# todo use dict
MAX_LEN = 300
PAD_LABEL = 3
BEG_LABEL = 1
IN_LABEL = 2
OUT_LABEL = 0

def load_datasets(dir_name):
    train_fname = 'train.tsv'
    test_fname = 'test.tsv'
    dev_fname = 'devel.tsv'
    res_ds = []
    for fname in [train_fname, test_fname, dev_fname]:
        x, y, cur_sent_x, cur_sent_y =[], [], [], []
        with open(dir_name+fname) as file:
            for line in file:
                if line != '\n':
                    word, label = line.split('\t')
                    cur_sent_x.append(word)
                    cur_sent_y.append(label[:-1])
                else:
                    y.append(cur_sent_y)
                    x.append(cur_sent_x)
                    cur_sent_x, cur_sent_y = [], []
        res_ds.append([x, y]) 
    return res_ds

def prepare_dataset(ds, tokenizer):
    x, y = ds
    res_x, res_y = [], []
    for ids in range(len(x)):
        new_sent, new_labels = [tokenizer.convert_tokens_to_ids(['[CLS]'])[0]], [0]
        sent, labels = x[ids], y[ids]
        for idp in range(len(sent)):
            tokens = tokenizer.tokenize(sent[idp].lower())
            tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
            new_sent += tokens_ids
            if labels[idp] == 'O':
                cur_label = OUT_LABEL
            elif labels[idp] == 'I-GENE':
                cur_label = IN_LABEL
            elif labels[idp] == 'B-GENE':
                cur_label = BEG_LABEL

            if cur_label == BEG_LABEL:
                new_labels += [cur_label] 
                new_labels += [IN_LABEL] * (len(tokens_ids) - 1)
            else:
                new_labels += [cur_label] *  len(tokens_ids)

        new_sent.append(tokenizer.convert_tokens_to_ids(['[SEP]'])[0])
        new_labels.append(0)
        while len(new_sent) < MAX_LEN:
            new_sent.append(0)
            new_labels.append(PAD_LABEL) #add padding
        res_x.append(new_sent)
        res_y.append(new_labels)
    return [res_x, res_y]   



tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
r =load_datasets(dir_name)
dsets = []
for ds in r:
    prepared = prepare_dataset(ds, tokenizer)
    dsets.append(prepared)



In [4]:
# utils
def get_ent_list(seq):
    res = []
    start_ent = 0
    for i in range(len(seq)):
        if seq[i] == BEG_LABEL:
            if start_ent != 0:
                res.append((start_ent-1, i-1))
            start_ent = i+1
        elif seq[i] == OUT_LABEL and start_ent != 0:
            res.append((start_ent-1, i-1))
            start_ent = 0
    if start_ent != 0:
        res.append((start_ent-1, len(seq)-1))
    return res


def ner_get_prep_vectors(t, p):
    true_ent, pred_ent = set(get_ent_list(t)), set(get_ent_list(p))
    vector_len = len(true_ent.union(pred_ent))
    true_vector = [1] * len(true_ent) + [0] * (vector_len - len(true_ent))
    pred_vector = [0] * (vector_len - len(pred_ent)) + [1] * len(pred_ent)
    return true_vector, pred_vector


In [5]:
def fit(model, x, tags, n_epochs, lr, batch_size=32):
    masks = [[float(i != 0.0) for i in ii] for ii in x]

    t_inputs = torch.tensor(x)
    t_tags = torch.tensor(tags)
    t_masks = torch.tensor(masks)

    train_data = TensorDataset(t_inputs, t_masks, t_tags)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.cuda()

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
    
    optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=lr,
    eps=1e-6)

    epochs = n_epochs
    max_grad_norm = 1.5

    # Total number of training steps is number of batches * number of epochs.
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler.


    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0.1 * total_steps,
        num_training_steps=total_steps)
    
    model.train()
    for _ in trange(epochs, desc="Epoch"):
        total_loss = 0

        for step, batch in enumerate(train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            model.zero_grad()
        
            outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        
            loss = outputs[0]
            loss.backward()
            total_loss += loss.item()

            #torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
            optimizer.step()
            scheduler.step()


def predict(model, x, true_tags, batch_size=32):
    masks = [[float(i != 0.0) for i in ii] for ii in x]

    t_inputs = torch.tensor(x)
    t_tags = torch.tensor(true_tags)
    t_masks = torch.tensor(masks)

    test_data = TensorDataset(t_inputs, t_masks, t_tags)
    test_sampler = RandomSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []

    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
            
        logits = outputs[1].detach().cpu().numpy()

        label_ids = b_labels.to('cpu').numpy()

        eval_loss += outputs[0].mean().item()

        batch_labels = [list(p) for p in np.argmax(logits, axis=2)]
        for i in range(len(label_ids)):
            sent_end = np.where(label_ids[i] == PAD_LABEL)[0][0]
            true_s, pred_s = label_ids[i][:sent_end], batch_labels[i][:sent_end] 
            proc_true, proc_pred = ner_get_prep_vectors(true_s, pred_s)
            predictions.extend(proc_pred)
            true_labels.extend(proc_true)


    print("Validation F1-Score: {}".format(f1_score(predictions, true_labels)))
    return predictions, true_labels




In [7]:
from transformers import BertForTokenClassification

model = BertForTokenClassification.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
    num_labels=4,
    output_attentions = False,
    output_hidden_states = False)
x_train, y_train = dsets[0]

fit(model, x_train, y_train, n_epochs=5, lr=3e-5)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForToken

In [8]:
x_test, y_test = dsets[1]
pred_tags, true_tags = predict(model, x_test, y_test)

Validation F1-Score: 0.8387963036095095


In [11]:
model = BertForTokenClassification.from_pretrained("dmis-lab/biobert-v1.1",
    num_labels=4,
    output_attentions = False,
    output_hidden_states = False)
x_train, y_train = dsets[0]

fit(model, x_train, y_train, n_epochs=5, lr=3e-5)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch: 100%|██████████| 5/5 [58:44<00:00, 704.83s/it]


In [12]:
x_test, y_test = dsets[1]
pred_tags, true_tags = predict(model, x_test, y_test)

Validation F1-Score: 0.8120040851598711


In [ ]:
diedevice

device(type='cuda')

In [6]:
model = BertForTokenClassification.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
    num_labels=4,
    output_attentions = False,
    output_hidden_states = False)
x_train, y_train = dsets[0]
 
fit(model, x_train, y_train, n_epochs=10, lr=5e-5)
x_test, y_test = dsets[1]
pred_tags, true_tags = predict(model, x_test, y_test)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForToken

Validation F1-Score: 0.8370189659242937
